# Arkhamdb Decklists Data Loading 
Created: 2021-10-29  
Updated: 2022-02-23  
Author: Spencer Simon

## Overview

This notebook downloads data from [arkhamdb.com](https://arkhamdb.com) using their [public api](https://arkhamdb.com/api/).  

The data downloaded includes all decklists created and published on the site.  

This data is downloaded and prepared for use in creating racing bar charts of investigator popularity over time, as well as additional analysis.  

Data is exported as a CSV.

## Setup

### Install and import necessary libraries

In [1]:
import pandas as pd
import urllib.request, json 
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import time

### Define global variables

In [2]:
base_url = 'https://arkhamdb.com'
api_deck_by_date = '/api/public/decklists/by_date/'

In [3]:
# Get current year and month as ints and strings of numbers:
current_yr = datetime.now().year
current_yr_string = str(current_yr)
current_month = datetime.now().month

# Make sure month string is 2 digits
if current_month <= 9:
    current_month_string = '0' + str(current_month)
else:
    current_month_string = str(current_month)

In [4]:
# Hard code specific dates for 1st 2 months, when very few decks were created
dates_old = ['2016-09-02', '2016-10-12', '2016-10-15', '2016-10-18', '2016-10-19', 
             '2016-10-22', '2016-10-23', '2016-10-24', '2016-10-27', '2016-10-29', 
             '2016-10-30']
urls_old = [base_url + api_deck_by_date + i for i in dates_old]

### Define functions

In [5]:
def get_urls(year, month):
    """
    year: string (YYYY)
    month: string (MM)
    
    Returns a list of the arkhamdb API urls for all decklists from all days in the input year and month.
    Includes some addition days/urls as well (e.g. February 30th).
    """
    
    # Check that input parameters are strings, and month is length 2
    if not isinstance(year, str) or not isinstance(month, str):
        print("Error: year and month must be strings")
        return
    elif len(month) != 2:
        print("Error: input month must be 2 characters long (e.g. 02) /n")
        return
    
    urls = []
    # Create list of url strings I will use to request data
    for i in range(1,32):
        if i <= 9:
            url_temp = base_url + api_deck_by_date + f'{year}-{month}' + '-0' + str(i)
        else:
            url_temp = base_url + api_deck_by_date + f'{year}-{month}-' + str(i)
        urls.append(url_temp)
    
    return urls

In [6]:
def fill_df_month(year, month):
    """
    year: string (YYYY)
    month: string (MM)
    
    Returns a list with arkhamdb information retrieved from the API for decklists for the given month.
    """
    
    # Check that input parameters are strings, and month is length 2
    if not isinstance(year, str) or not isinstance(month, str):
        print("Error: year and month must be strings")
        return
    elif len(month) != 2:
        print("Error: input month must be 2 characters long (e.g. 02) /n")
        return
    
    small_dfs = []
    
    # Set a special case for September and October 2016, when very few decks were published
    if year == '2016' and month == '09':         # Fill using hard coded values for first day from urls_old, the only day in September
        try:
            small_dfs.append(pd.read_json(urls_old[0]))
        except:
            print(f"Error for Date {url[-10:]} \n")
    elif year == '2016' and month == '10':
        for url in urls_old[1:]:             # Fill using rest of urls_old, which is october
            try:
                small_dfs.append(pd.read_json(url))
            except:
                print(f"Error for Date {url[-10:]} \n")
    else:
        for url in get_urls(year, month):    # Else, fill in list of decklists for the month using get_urls()
            try:
                small_dfs.append(pd.read_json(url))
            except:
                print(f"Error for Date {url[-10:]} \n")

    return small_dfs

In [7]:
def fill_df_full():
    """
    Returns a dataframe with all arkhamdb data from the arkhamdb API for decklists through the previous month.
    """
    month_lists = [] # Initialize list of decklists
    
    # Loop through all years 2016 to present
    for yr in range(2016, current_yr+1):
        # Set start month to 9 in 2016, as that is the first month a decklist is published
        if yr == 2016:
            month_iter = ['09', '10', '11', '12']
        else:
            month_iter = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
            
        # Print when starting a new year to update progress:
        print(f" ------------------ \n Starting year {yr} \n")
        
        # Loop through all months through the month before the current month, 
        # so we don't include partial month data for this month
        # For each month, get all decklists for the month. Append that to overall months list.
        for m in month_iter:
            if yr == current_yr and m == current_month_string:
                break    # End the loop if it's the current month
            temp_list = fill_df_month(str(yr), m)
            month_lists.append(temp_list)         # Append to lists here, then convert to df later
    
    # flatten list so I can pd.concat to make df
    flatter_list = [item for sublist in month_lists for item in sublist] 
    
    df = pd.concat(flatter_list, ignore_index=True)    # Create dataframe from the list above
    df.set_index(['id'], inplace=True)                 # set df index to id column
    
    return df

## Query Arkhamdb to get data

In [8]:
# Initially: took ~25 mins to run. Look at speed improvements, multi-processing & multi-threading, etc.
df_decklists_raw = fill_df_full()

 ------------------ 
 Starting year 2016 

Error for Date 2016-11-02 

Error for Date 2016-11-04 

Error for Date 2016-11-09 

Error for Date 2016-11-27 

Error for Date 2016-12-08 

Error for Date 2016-12-09 

Error for Date 2016-12-24 

Error for Date 2016-12-25 

Error for Date 2016-12-28 

Error for Date 2016-12-31 

 ------------------ 
 Starting year 2017 

Error for Date 2017-01-09 

Error for Date 2017-01-11 

Error for Date 2017-01-13 

Error for Date 2017-02-19 

Error for Date 2017-04-31 

Error for Date 2017-05-01 

Error for Date 2017-07-28 

Error for Date 2017-07-29 

Error for Date 2017-08-26 

Error for Date 2017-12-23 

 ------------------ 
 Starting year 2018 

Error for Date 2018-05-07 

 ------------------ 
 Starting year 2019 

 ------------------ 
 Starting year 2020 

 ------------------ 
 Starting year 2021 

 ------------------ 
 Starting year 2022 



## Clean Data

In [9]:
df_decklists_raw.head()

,name,date_creation,date_update,description_md,user_id,investigator_code,investigator_name,slots,sideSlots,ignoreDeckLimitSlots,version,xp,xp_adjustment,exile_string,taboo_id,meta,tags,previous_deck,next_deck
id,,,,,,,,,,,,,,,,,,,
1,Roland Banks Starter Deck,2016-09-02T13:17:02+00:00,2022-01-15T19:13:07+00:00,Roland Banks suggested deck from the Core Set\...,3,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 1, '01017': ...",[],NaN,1,NaN,NaN,NaN,NaN,,,NaN,NaN
2,Wendy Adams Starter Deck,2016-09-02T13:18:34+00:00,2022-02-10T17:12:27+00:00,Wendy Adams starter deck from the core set.,3,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,1,NaN,NaN,NaN,NaN,,,NaN,NaN
3,I'm Half Crazy,2016-10-12T07:59:47+00:00,2016-11-19T09:07:28+00:00,Deck built with 2x Core Set and before all car...,113,1002,Daisy Walker,"{'01008': 1, '01009': 1, '01030': 2, '01031': ...",[],NaN,1,NaN,NaN,NaN,NaN,,,NaN,NaN
4,"Roland Banks, Just a G-Man",2016-10-12T16:50:58+00:00,2020-04-20T04:16:32+00:00,There are likely two main ways to build [Rolan...,70,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 2, '01017': ...",[],NaN,1,NaN,NaN,NaN,NaN,,,NaN,NaN
6,Orphan's First deck,2016-10-15T22:39:06+00:00,2018-07-09T02:05:33+00:00,My first deck!\r\n\r\nPlan is that this will l...,148,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,1,NaN,NaN,NaN,NaN,,,NaN,NaN


In [10]:
df_decklists_raw.shape

(32007, 19)

In [11]:
df_decklists_raw.isnull().sum()

name                        0
date_creation               0
date_update                 0
description_md              0
user_id                     0
investigator_code           0
investigator_name           0
slots                       0
sideSlots                   0
ignoreDeckLimitSlots    31848
version                     0
xp                      21192
xp_adjustment           10525
exile_string            31887
taboo_id                25308
meta                        0
tags                        0
previous_deck           21192
next_deck               21192
dtype: int64

In [12]:
# Copy raw dataframe to work with
df_decklists_clean = df_decklists_raw.copy()

In [13]:
# Clean up creation dates
df_decklists_clean['date_creation'] = pd.to_datetime(df_decklists_clean['date_creation'],
                                                     format='%Y-%m-%dT%H:%M:%S+00:00')

df_decklists_clean['create_year'] = df_decklists_clean['date_creation'].dt.year
df_decklists_clean['create_month'] = df_decklists_clean['date_creation'].dt.month
df_decklists_clean['create_day'] = df_decklists_clean['date_creation'].dt.day

In [14]:
df_decklists_clean.head()

,name,date_creation,date_update,description_md,user_id,investigator_code,investigator_name,slots,sideSlots,ignoreDeckLimitSlots,...,xp_adjustment,exile_string,taboo_id,meta,tags,previous_deck,next_deck,create_year,create_month,create_day
id,,,,,,,,,,,,,,,,,,,,,
1,Roland Banks Starter Deck,2016-09-02 13:17:02,2022-01-15T19:13:07+00:00,Roland Banks suggested deck from the Core Set\...,3,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 1, '01017': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,9,2
2,Wendy Adams Starter Deck,2016-09-02 13:18:34,2022-02-10T17:12:27+00:00,Wendy Adams starter deck from the core set.,3,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,9,2
3,I'm Half Crazy,2016-10-12 07:59:47,2016-11-19T09:07:28+00:00,Deck built with 2x Core Set and before all car...,113,1002,Daisy Walker,"{'01008': 1, '01009': 1, '01030': 2, '01031': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,12
4,"Roland Banks, Just a G-Man",2016-10-12 16:50:58,2020-04-20T04:16:32+00:00,There are likely two main ways to build [Rolan...,70,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 2, '01017': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,12
6,Orphan's First deck,2016-10-15 22:39:06,2018-07-09T02:05:33+00:00,My first deck!\r\n\r\nPlan is that this will l...,148,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,15


In [15]:
# Create smaller df w/ relevant columns for investigator racing bar chart 
#df_investigator_pop_start = df_decklists_clean[["name", "date_creation", 
#                                                "investigator_code", "investigator_name",
#                                                "create_year", "create_month", "create_day"]]

In [16]:
#df_investigator_pop_start.tail()

### Remove Duplicates

In [17]:
# View Duplicates
#df_investigator_pop_start.duplicated()

In [18]:
# View Duplicates on X colmns
df_decklists_clean.duplicated(subset=['name', 'date_creation', 'investigator_code'])

id
1        False
2        False
3        False
4        False
6        False
         ...  
35698    False
35699     True
35700     True
35701     True
35702     True
Length: 32007, dtype: bool

In [20]:
# Number of duplicates and Non-duplicates
print(df_decklists_clean.duplicated(subset=['name', 'date_creation', 'investigator_code']).sum())
print((~df_decklists_clean.duplicated(subset=['name', 'date_creation', 'investigator_code'])).sum())

11022
20985


In [21]:
# Extract duplicate rows (showing all duplicates)
df_decklists_clean.loc[df_decklists_clean.duplicated(subset=['name', 'date_creation', 'investigator_code'],
                                                                   keep=False), :]

,name,date_creation,date_update,description_md,user_id,investigator_code,investigator_name,slots,sideSlots,ignoreDeckLimitSlots,...,xp_adjustment,exile_string,taboo_id,meta,tags,previous_deck,next_deck,create_year,create_month,create_day
id,,,,,,,,,,,,,,,,,,,,,
158,[Multiplayer] Daisy Walkers the Walk & Talkers...,2016-12-01 17:34:25,2016-12-01T17:34:25+00:00,*It should be noted that this is a deck built ...,1082,1002,Daisy Walker,"{'01008': 1, '01009': 1, '01030': 1, '01031': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,12,1
159,Daisy Walker: Book Warrior (Ecom/Clue-gen),2016-12-01 21:50:34,2016-12-01T21:50:34+00:00,"Built to have a late game swing, this Daisy Wa...",1112,1002,Daisy Walker,"{'01000': 1, '01008': 1, '01009': 1, '01030': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,12,1
161,Solo Campaign Roland Banks,2016-12-01 22:29:45,2017-07-12T03:51:43+00:00,Constructed for The Devourer Below. Began wit...,454,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 1, '01017': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,12,1
158,[Multiplayer] Daisy Walkers the Walk & Talkers...,2016-12-01 17:34:25,2016-12-01T17:34:25+00:00,*It should be noted that this is a deck built ...,1082,1002,Daisy Walker,"{'01008': 1, '01009': 1, '01030': 1, '01031': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,12,1
159,Daisy Walker: Book Warrior (Ecom/Clue-gen),2016-12-01 21:50:34,2016-12-01T21:50:34+00:00,"Built to have a late game swing, this Daisy Wa...",1112,1002,Daisy Walker,"{'01000': 1, '01008': 1, '01009': 1, '01030': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35698,Carolyn's dreamy synergy,2022-01-31 22:29:08,2022-02-10T01:40:25+00:00,[This deck is largely a copy of this one](http...,4145,5001,Carolyn Fern,"{'01058': 2, '01065': 1, '01091': 1, '02185': ...","{'08062': 2, '08114': 1, '08115': 1, '08117': ...",NaN,...,0.0,NaN,4.0,{},,35699.0,NaN,2022,1,31
35699,Carolyn's dreamy synergy,2022-01-31 22:29:08,2022-01-31T22:29:08+00:00,[This deck is largely a copy of this one](http...,4145,5001,Carolyn Fern,"{'01058': 2, '01065': 1, '01091': 1, '02185': ...","{'08062': 2, '08114': 1, '08115': 1, '08117': ...",NaN,...,0.0,NaN,4.0,{},,35700.0,35698.0,2022,1,31
35700,Carolyn's dreamy synergy,2022-01-31 22:29:08,2022-01-31T22:29:08+00:00,[This deck is largely a copy of this one](http...,4145,5001,Carolyn Fern,"{'01058': 2, '01065': 1, '01091': 2, '02030': ...","{'05280': 2, '08022': 1, '08062': 2, '08114': ...",NaN,...,0.0,NaN,4.0,{},,35701.0,35699.0,2022,1,31


In [22]:
# Drop Duplicates. Keep 1st occurence and drop all others
df_decklists_clean = df_decklists_clean.drop_duplicates(subset=['name', 'date_creation', 'investigator_code'])

In [23]:
df_decklists_clean.shape

(20985, 22)

In [24]:
df_decklists_clean.head()

,name,date_creation,date_update,description_md,user_id,investigator_code,investigator_name,slots,sideSlots,ignoreDeckLimitSlots,...,xp_adjustment,exile_string,taboo_id,meta,tags,previous_deck,next_deck,create_year,create_month,create_day
id,,,,,,,,,,,,,,,,,,,,,
1,Roland Banks Starter Deck,2016-09-02 13:17:02,2022-01-15T19:13:07+00:00,Roland Banks suggested deck from the Core Set\...,3,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 1, '01017': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,9,2
2,Wendy Adams Starter Deck,2016-09-02 13:18:34,2022-02-10T17:12:27+00:00,Wendy Adams starter deck from the core set.,3,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,9,2
3,I'm Half Crazy,2016-10-12 07:59:47,2016-11-19T09:07:28+00:00,Deck built with 2x Core Set and before all car...,113,1002,Daisy Walker,"{'01008': 1, '01009': 1, '01030': 2, '01031': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,12
4,"Roland Banks, Just a G-Man",2016-10-12 16:50:58,2020-04-20T04:16:32+00:00,There are likely two main ways to build [Rolan...,70,1001,Roland Banks,"{'01006': 1, '01007': 1, '01016': 2, '01017': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,12
6,Orphan's First deck,2016-10-15 22:39:06,2018-07-09T02:05:33+00:00,My first deck!\r\n\r\nPlan is that this will l...,148,1005,Wendy Adams,"{'01014': 1, '01015': 1, '01044': 1, '01045': ...",[],NaN,...,NaN,NaN,NaN,,,NaN,NaN,2016,10,15


## Export Data

In [25]:
timestr = time.strftime("%Y-%m-%d")
filestr_raw = 'decklists_raw_'+timestr+'.csv'
filestr_cln = 'decklists_clean_'+timestr+'.csv'
print(filestr_raw)

decklists_raw_2022-02-24.csv


In [26]:
df_decklists_raw.to_csv(filestr_raw)
df_decklists_clean.to_csv(filestr_cln)

In [ ]:
#df_inv_pop_final.to_csv('investigator_popularity_raw.csv')

## Notes & To-Do's:

- Current function takes a long time to run. Use multithreading or other improvements to make faster
- Write list of known errors/improvements

### Tests for future work: 

In [ ]:
#with urllib.request.urlopen(url_deck) as url:
#    data = json.loads(url.read().decode())
#    #print(json.dumps(data, indent=1))

In [ ]:
#def get_url(url):
#    return requests.get(url)

#list_of_urls = ["https://postman-echo.com/get?foo1=bar1&foo2=bar2"]*10

#with ThreadPoolExecutor(max_workers=2) as pool:
#    response_list = list(pool.map(get_url,list_of_urls))

#for response in response_list:
#    print(response)